Загрузка библиотек,файлов и создание функции для обработки

In [1]:
import pandas as pd #импорт библеотеки 
from sklearn.neighbors import KNeighborsClassifier #думал, что молодец и сделаю крутую модель для кластерации всех msisdn =)(как я ошибался)
import datetime #импорт для работы с форматом даты

In [61]:
#загрузил файлы
facts=pd.read_excel('01_Facts.xlsx')
event=pd.read_excel('04_event_type.xlsx')
devices=pd.read_csv('03_устройства.csv',sep=',""',encoding='utf-8',decimal=',',engine='python')
data_test=pd.read_csv('02_Data_test.csv',sep=';',encoding='utf-8',decimal=",",engine='python')

In [57]:
#Немного хаотичного форматирования 
devices.rename(columns={'"tac':'tac','vendor""':'vendor','platform""':'platform','type"""':'type'},inplace=True)
devices1=devices[['tac','vendor','platform','type']].replace({'\"':''},regex=True)
print(devices1.info())
devices1.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73263 entries, 0 to 73262
Data columns (total 4 columns):
tac         73263 non-null object
vendor      73263 non-null object
platform    73263 non-null object
type        73263 non-null object
dtypes: object(4)
memory usage: 2.2+ MB
None


,tac,vendor,platform,type
0,00100429,PROTO,Proprietary,Phone
1,00100430,PROTO,Proprietary,Phone
2,00100431,PROTO,Proprietary,Phone
3,00100432,PROTO,Proprietary,Phone
4,00100433,PROTO,Proprietary,Phone


In [33]:
#Функция, которая получает timestamp и обработно выводит в формате "2013-25-03" 
def time(x):
    value = datetime.datetime.fromtimestamp(x/1000)
    return(value.strftime('%Y-%m-%d'))#'%Y-%m-%d-%H.%M.%S' полный формат даты

In [59]:
#форматирование входящего Датафрейма 
def prepoc(df_input):
    df_output=df_input.copy()
    #применяем функцию сверху
    df_output['date']=df_output['tstamp'].apply(time)
    #приводим к строчным данным
    df_output[['cid','lat','long','date']]=df_output[['cid','lat','long','date']].astype(str)
    #соединил место и время соединения(думал, что так можно будет сделать уникальное значение где ИД абонентов пересекаются)
    df_output['date_place']=df_output[['long','lat','date']].apply(lambda x:'-'.join(x), axis=1) 
    #удалил все лишнее(чтобы сразу загрузить данные в модель). По крайней мере я так думал =)
    df_output=df_output.drop(['long','lat','date','lac','cid','imei','event_type','max_dist','cell_type','start_angle','end_angle','tstamp'],axis=1)
    
    return df_output

In [62]:
data_view=data_test.pipe(prepoc)#применил функцию на датафрейм
print(data_view.info())
data_view.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174123 entries, 0 to 174122
Data columns (total 2 columns):
msisdn        174123 non-null int64
date_place    174123 non-null object
dtypes: int64(1), object(1)
memory usage: 2.7+ MB
None


,msisdn,date_place
0,158529599791,37.9305-55.7815-2013-05-22
1,158520145943,37.4239-55.7426-2013-05-22
2,158521798391,37.7034-55.5803-2013-05-22
3,158537830573,37.7895-55.7042-2013-05-22
4,158510204039,37.19-55.8308-2013-05-22


In [15]:
data=data_view.groupby('msisdn').count()
print(len(data[data['date_place']<2]))

31


сохранение в файл


In [57]:
%%time
#Сюда записал бы результат работы 
writer = pd.ExcelWriter('data1.xlsx', engine='xlsxwriter')
data.to_excel(writer,sheet_name='1')

Wall time: 268 ms
